<a href="https://colab.research.google.com/github/Thayyebsalim/music_analyisis/blob/master/Music_Sentimental_analysis_NLP_by_Tone_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The idea of this code is to find the mood of the song in any way possible helping us to segregate songs better.

1.   First part is gonna be scraping the lyrics from any website or API 
2.   Finding the vibe of the song with sentimental analysis
3.   Fetch and mp3 version and check the mood with transfer learning hopefully or something similar



In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (145 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("vader_lexicon")
nltk.download("brown")
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We need to scrape the song lyrics from websites or API to use here

I am using selenium here and the speed is not the best at the moment so I might switch to scrapy or use an API

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


def get_track(): # Gets track name from user and converts it so that we can use it in a link
  track_name = input("Enter a track name: ")
  track_name_link = track_name.rstrip().replace(" ","%20").lower()
  return track_name_link


def scrape_tracklist(track_link): # Scrapes the page for list of tracks with similar names or such 
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get("https://www.musixmatch.com/search/" + track_link + "/tracks")
  element = driver.find_element_by_class_name("box-content")
  link_list = [i.get_attribute('href') for i in element.find_elements_by_tag_name('a')]
  driver.close()
  lyric_links=[i for i in link_list if re.search("/lyrics/",i)] #could do it in one line but for the sake of clarity, I'm gonna do two
  lyric_links=[i for i in lyric_links if not re.search("/add",i)]
  return lyric_links


def scrape_lyrics(lyric_links_list,choice): # Scrapes the lyrics element
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get(lyric_links_list[choice])
  lyric_elem=driver.find_elements_by_class_name('mxm-lyrics__content')
  lyric_elem= [i.find_element_by_tag_name('span') for i in lyric_elem] 
  lyric_elem= [i.get_attribute('innerHTML') for i in lyric_elem]
  driver.close()
  return lyric_elem


def select_track(artist_track_list): # Lets the user select a track
  sel=0
  for track in artist_track_list:
    print(f"{track[0]+1} {track[1]} - {track[2]}")
  sel=int(input('Enter the number of your desired track: '))-1
  return sel


def get_lyric_element(tracklist,choice=0): # Cleans up the lyric element and returns a cleaned list(array) with just lyrics inside
  lyric_links_list = scrape_tracklist(track_link)
  artist_track_list = [[lyric_links_list.index(x), x.split("/")[4],x.split('/')[5]] for x in lyric_links_list]
  print(artist_track_list)
  if len(artist_track_list) > 1 and choice != "default":
    choice = select_track(artist_track_list)
  elif len(artist_track_list) == 0 :
    return "Sorry! no such song"
  else:
    choice=0
  lyric_element = scrape_lyrics(lyric_links_list,choice)
  return " ".join(lyric_element)



In [4]:
track_name= "let me down slowly"
track_link=track_name.rstrip().replace(" ","%20").lower()
#track_link=get_track() #uncomment to activate function

In [6]:
lyric_element = get_lyric_element(track_link,"default")
lyric_element

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


[[0, 'Alec-Benjamin', 'Let-Me-Down-Slowly'], [1, 'Alec-Benjamin-feat-Alessia-Cara', 'Let-Me-Down-Slowly'], [2, 'Dj-Mgebahi-feat-Alec-Benjamin', 'Let-Me-Down-Slowly'], [3, 'DMNDS-Fets-Poky', 'Let-Me-Down-Slowly'], [4, 'Kurt-Hugo-Schneider-feat-First-to-Eleven', 'Let-Me-Down-Slowly'], [5, 'Gavin-Mikhail-feat-The-Cameron-Collective', 'Let-Me-Down-Slowly-Acoustic'], [6, 'Arcade-Player', 'Let-Me-Down-Slowly-16-Bit-Alec-Benjamin-Alessia-Cara-Emulation'], [7, 'Adam-Christopher-feat-Dan-Berk', 'Let-Me-Down-Slowly-Acoustic'], [8, 'Fly-By-Nightcore', 'Let-Me-Down-Slowly-Switching-Vocals'], [9, 'Beth', 'Let-Me-Down-Slowly-Acoustic'], [10, 'Powell-Sullivan', 'Let-Me-Down-Slowly'], [11, 'Alex-Goot-feat-Jada-Facer', 'Let-Me-Down-Slowly'], [12, 'Ben-Woodward-5', 'Let-Me-Down-Slowly-Acoustic-Piano'], [13, 'Nightcore-God', 'Let-Me-Down-Slowly'], [14, 'Cyan-Kicks', 'Let-Me-Down-Slowly'], [15, 'Sapphire-10', 'Let-Me-Down-Slowly'], [16, 'RESE-4', 'Let-Me-Down-Slowly'], [17, 'Myah', 'Let-Me-Down-Slowly'], 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


"This night is cold in the kingdom\nI can feel you fade away\nFrom the kitchen to the bathroom sink and Your steps keep me awake\n\nDon't cut me down, throw me out, leave me here to waste\nI once was a man with dignity and grace\nNow I'm slipping through the cracks of your cold embrace\nSo please, please\n\nCould you find a way to let me down slowly?\nA little sympathy, I hope you can show me\nIf you wanna go, then I'll be so lonely\nIf you're leaving, baby, let me down slowly\nLet me down, down\nLet me down, down\nLet me down, let me down\nDown, let me down, down\nLet me down\nIf you wanna go, then I'll be so lonely\nIf you're leaving, baby, let me down slowly\n\nCold skin, drag my feet on the tile\nAs I'm walking down the corridor\nAnd I know we haven't talked in a while\nSo I'm looking for an open door\n\nDon't cut me down, throw me out, leave me here to waste\nI once was a man with dignity and grace\nNow I'm slipping through the cracks of your cold embrace\nSo please, please\n\nCou

The first part is done, web scraper should work fine. I did not implement error cases for now becuase I want to focus on the next two parts of the project


In [7]:
def filter_raw_lyrics(raw_lyr): #The scraped element has blanks and \n's lets filter those 
  return [i for i in raw_lyr.split("\n") if i != '']

#joiner
def convert_arr_to_str(arr):
  return " ".join(arr)

In [8]:
lyrics_as_array= filter_raw_lyrics(lyric_element)
lyrics=convert_arr_to_str(lyrics_as_array) #returns a single string as lyrics
lyrics,lyrics_as_array

("This night is cold in the kingdom I can feel you fade away From the kitchen to the bathroom sink and Your steps keep me awake Don't cut me down, throw me out, leave me here to waste I once was a man with dignity and grace Now I'm slipping through the cracks of your cold embrace So please, please Could you find a way to let me down slowly? A little sympathy, I hope you can show me If you wanna go, then I'll be so lonely If you're leaving, baby, let me down slowly Let me down, down Let me down, down Let me down, let me down Down, let me down, down Let me down If you wanna go, then I'll be so lonely If you're leaving, baby, let me down slowly Cold skin, drag my feet on the tile As I'm walking down the corridor And I know we haven't talked in a while So I'm looking for an open door Don't cut me down, throw me out, leave me here to waste I once was a man with dignity and grace Now I'm slipping through the cracks of your cold embrace So please, please Could you find a way to let me down sl

Well lets have some fun with the lyrics

In [9]:
def swfilter(word): #function we use in filter()
  return word not in stopwords.words()

def remove_stopword(lyr_arr):
  return [" ".join(filter(swfilter,sentence.split())) for sentence in lyrics_as_array]

In [10]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def map_contraction(word): #function we use in map()
  word=word.lower()
  if word in contractions:
    return contractions[word]
  else: 
    return word

def remove_contraction(lyr_arr):
  return [" ".join(map(map_contraction, sentence.split())) for sentence in lyr_arr]

In [11]:
wordnet_lemmatizer = WordNetLemmatizer()

def map_lemmatize(word):
  return wordnet_lemmatizer.lemmatize(word, pos="v")

def lemmatize_words(lyr_arr):
  return [" ".join(map(map_lemmatize, sentence.split())) for sentence in lyr_arr]

In [12]:
lyrics_as_array_without_sw = remove_stopword(lyrics_as_array)
lyrics_without_sw = convert_arr_to_str(lyrics_as_array_without_sw)
lyrics_as_array_without_sw,lyrics_without_sw

(['This night cold kingdom',
  'I feel fade away',
  'From kitchen bathroom sink Your steps keep awake',
  "Don't cut down, throw out, leave waste",
  'I dignity grace',
  "Now I'm slipping cracks cold embrace",
  'So please, please',
  'Could find way let slowly?',
  'A little sympathy, I hope show',
  "If wanna go, I'll lonely",
  'If leaving, baby, let slowly',
  'Let down,',
  'Let down,',
  'Let down, let',
  'Down, let down,',
  'Let',
  "If wanna go, I'll lonely",
  'If leaving, baby, let slowly',
  'Cold skin, drag feet',
  "As I'm walking corridor",
  'And I know talked',
  "So I'm looking open",
  "Don't cut down, throw out, leave waste",
  'I dignity grace',
  "Now I'm slipping cracks cold embrace",
  'So please, please',
  'Could find way let slowly?',
  'A little sympathy, I hope show',
  "If wanna go, I'll lonely",
  'If leaving, baby, let slowly',
  'Let down,',
  'Let down,',
  'Let down, let',
  'Down, let down,',
  'Let',
  "If wanna go, I'll lonely",
  'If leaving, b

In [13]:
lyrics_as_array_without_contraction = remove_contraction(lyrics_as_array)
lyrics_without_contraction = convert_arr_to_str(lyrics_as_array_without_contraction)
lyrics_as_array_without_contraction,lyrics_without_contraction

(['this night is cold in the kingdom',
  'i can feel you fade away',
  'from the kitchen to the bathroom sink and your steps keep me awake',
  'do not cut me down, throw me out, leave me here to waste',
  'i once was a man with dignity and grace',
  "now i'm slipping through the cracks of your cold embrace",
  'so please, please',
  'could you find a way to let me down slowly?',
  'a little sympathy, i hope you can show me',
  "if you wanna go, then i'll be so lonely",
  'if you are leaving, baby, let me down slowly',
  'let me down, down',
  'let me down, down',
  'let me down, let me down',
  'down, let me down, down',
  'let me down',
  "if you wanna go, then i'll be so lonely",
  'if you are leaving, baby, let me down slowly',
  'cold skin, drag my feet on the tile',
  "as i'm walking down the corridor",
  'and i know we have not talked in a while',
  "so i'm looking for an open door",
  'do not cut me down, throw me out, leave me here to waste',
  'i once was a man with dignity an

In [14]:
lyrics_as_array_with_lemmatized_words=lemmatize_words(lyrics_as_array)
lyrics_with_lemmatized_words=convert_arr_to_str(lyrics_as_array_with_lemmatized_words)
lyrics_as_array_with_lemmatized_words,lyrics_with_lemmatized_words

(['This night be cold in the kingdom',
  'I can feel you fade away',
  'From the kitchen to the bathroom sink and Your step keep me awake',
  "Don't cut me down, throw me out, leave me here to waste",
  'I once be a man with dignity and grace',
  "Now I'm slip through the crack of your cold embrace",
  'So please, please',
  'Could you find a way to let me down slowly?',
  'A little sympathy, I hope you can show me',
  "If you wanna go, then I'll be so lonely",
  "If you're leaving, baby, let me down slowly",
  'Let me down, down',
  'Let me down, down',
  'Let me down, let me down',
  'Down, let me down, down',
  'Let me down',
  "If you wanna go, then I'll be so lonely",
  "If you're leaving, baby, let me down slowly",
  'Cold skin, drag my feet on the tile',
  "As I'm walk down the corridor",
  "And I know we haven't talk in a while",
  "So I'm look for an open door",
  "Don't cut me down, throw me out, leave me here to waste",
  'I once be a man with dignity and grace',
  "Now I'm 


Next we have to tackle the sentimantal analysis part. We aim to try bag of words then use NLTK on every line might be slow but I like to give prominence to accuracy

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
def get_vader_score(lyr):
  # Polarity score returns dictionary
  sentiment = sia.polarity_scores(lyr)
  return sentiment['compound']
  """
  print(ss)
  print(ss['compound'])
  ss.pop("compound")
  print("ss=",ss)
  x=sorted(ss.values(), reverse=True)[0]
  for i in ss.keys():
    if ss[i]==x:
      if "i" == "neg":
        ss[i]=-(ss[i])
      print(ss[i])
  """  

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [16]:
get_vader_score(lyrics)

0.8804

In [17]:
def get_textblob_score(lyr):
  blob=TextBlob(lyr)
  return blob.polarity

In [18]:
get_textblob_score(lyrics)

-0.1845700152207

In [19]:
!pip install watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey="hzkkbjivKNHfoi3cFlMBOPG6-YHLc-7CAQKwXn8Rt7JT", 
    url="https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/512aa79c-a8ba-43fa-b9d5-a4382099b06f" # paste the url here
    )
def get_watson_score(input_text): 
    response = natural_language_understanding.analyze (
    text = input_text,
    features = Features(sentiment=SentimentOptions())).get_result()
    res = response.get('sentiment').get('document').get('score')
    return res

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  import sys


In [20]:
get_watson_score(lyrics)

-0.95373

In [21]:
sentiment_score_fns=[get_textblob_score,get_vader_score]

In [40]:
# Returns all transformed arrays and further maps the arrays 

def get_lyric_arrays(lyr_arr):
  return [lyr_arr,
          remove_stopword(lyr_arr),
          remove_contraction(lyr_arr),
          lemmatize_words(lyr_arr),
          remove_contraction(remove_stopword(lyr_arr)),
          lemmatize_words(remove_stopword(lyr_arr)),
          remove_contraction(lemmatize_words(lyr_arr)),
          lemmatize_words(remove_contraction(remove_stopword(lyr_arr)))]

def get_lyrics(arr):
  return list(map(convert_arr_to_str,arr))

list_of_converted_lyr_arr=get_lyric_arrays(lyrics_as_array) #returns a tuple with 3 items lyrics, lyrics without stop wordsm 
list_of_converted_lyr=get_lyrics(list_of_converted_lyr_arr)


In [41]:
import pandas as pd 
import numpy as np

In [66]:
df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df["Textblob"]=list(map(get_textblob_score,list_of_converted_lyr))
df["Vader"]=list(map(get_vader_score,list_of_converted_lyr))
df["Watson"]=list(map(get_watson_score,list_of_converted_lyr))
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,-0.184570,NaN,NaN,0.8804,NaN,NaN,-0.953730,NaN,NaN
SW,-0.204813,NaN,NaN,0.9737,NaN,NaN,-0.976958,NaN,NaN
CO,-0.184570,NaN,NaN,0.9150,NaN,NaN,-0.969718,NaN,NaN
LM,-0.186952,NaN,NaN,0.9272,NaN,NaN,-0.945934,NaN,NaN
SW-CO,-0.204813,NaN,NaN,0.9780,NaN,NaN,-0.984243,NaN,NaN
SW-LM,-0.209925,NaN,NaN,0.9673,NaN,NaN,-0.977691,NaN,NaN
CO-LM,-0.186952,NaN,NaN,0.9449,NaN,NaN,-0.965830,NaN,NaN
SW-CO-LM,-0.209925,NaN,NaN,0.9731,NaN,NaN,-0.984533,NaN,NaN


In [68]:
textblob_per_line=[]
for trans in list_of_converted_lyr_arr:
  textblob_per_line.append(list(map(get_textblob_score,trans)))

df["Textblob per line"]=list(map(np.mean,textblob_per_line))
textblob_per_line

[[-0.6,
  0.0,
  0.0,
  -0.1777777777777778,
  0.0,
  -0.35,
  0.0,
  -0.2277777777777778,
  -0.1875,
  -0.09999999999999998,
  -0.2277777777777778,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.09999999999999998,
  -0.2277777777777778,
  -0.35,
  -0.15555555555555559,
  0.0,
  0.0,
  -0.1777777777777778,
  0.0,
  -0.35,
  0.0,
  -0.2277777777777778,
  -0.1875,
  -0.09999999999999998,
  -0.2277777777777778,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.09999999999999998,
  -0.2277777777777778,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.2277777777777778,
  -0.1875,
  -0.09999999999999998,
  -0.2277777777777778,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.15555555555555559,
  -0.09999999999999998,
  -0.2277777777777778,

In [69]:
vader_per_line=[]
for trans in list_of_converted_lyr_arr:
  vader_per_line.append(list(map(get_vader_score,trans)))

df["Vader per line"]=list(map(np.mean,vader_per_line))
vader_per_line

[[0.0,
  0.0,
  0.0,
  -0.2928,
  0.6705,
  0.3182,
  0.6353,
  0.0,
  0.5898,
  -0.5009,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.5009,
  0.0,
  -0.2263,
  0.0,
  0.0,
  0.0,
  -0.2928,
  0.6705,
  0.3182,
  0.6353,
  0.0,
  0.5898,
  -0.5009,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.5009,
  0.0,
  0.0742,
  0.0742,
  0.0742,
  0.0742,
  0.0,
  0.5898,
  -0.5009,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.5009,
  0.0,
  -0.5009,
  0.0],
 [0.0,
  0.0,
  0.0,
  -0.2928,
  0.6705,
  0.3182,
  0.6353,
  0.0,
  0.5898,
  -0.3612,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.3612,
  0.0,
  -0.2263,
  0.0,
  0.0,
  0.0,
  -0.2928,
  0.6705,
  0.3182,
  0.6353,
  0.0,
  0.5898,
  -0.3612,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.3612,
  0.0,
  0.3252,
  0.3252,
  0.3252,
  0.3252,
  0.0,
  0.5898,
  -0.3612,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.3612,
  0.0,
  -0.3612,
  0.0],
 [0.0,
  0.0,
  0.0,
  -0.2928,
  0.6705,
  0.3182,
  0.6353,
  0.0,
  0.5898,
  -0.

In [70]:
def watson_min_words_arr(arr): #watson has minimum requirement to perform sentiment analysis
  x="" 
  watson_arr=[]
  for line in arr:
    if len(line.split())<4:
      x=x+line
      continue
    if x!= "" : line=x+" "+line
    x=""
    watson_arr.append(line)
  return watson_arr 

converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))

In [71]:
watson_per_line=[]
for trans in converted_watson_arr:
  watson_per_line.append(list(map(get_watson_score,trans)))

df["Watson per line"]=list(map(np.mean,watson_per_line))
watson_per_line

[[0,
  -0.815836,
  -0.359449,
  -0.958112,
  0.956868,
  -0.800306,
  0,
  0.478253,
  -0.931047,
  -0.737152,
  -0.823413,
  -0.823413,
  -0.89039,
  -0.849114,
  -0.924073,
  -0.737152,
  -0.826445,
  0,
  0,
  0,
  -0.958112,
  0.956868,
  -0.800306,
  0,
  0.478253,
  -0.931047,
  -0.737152,
  -0.823413,
  -0.823413,
  -0.89039,
  -0.849114,
  -0.924073,
  -0.737152,
  -0.737313,
  -0.737313,
  -0.737313,
  -0.737313,
  0,
  0.478253,
  -0.931047,
  -0.737152,
  -0.823413,
  -0.823413,
  -0.89039,
  -0.849114,
  -0.924073,
  -0.737152,
  -0.931047,
  -0.737152],
 [0,
  -0.871772,
  0,
  -0.62017,
  0.42122,
  0,
  0.732301,
  -0.915224,
  0,
  -0.999682,
  0,
  -0.943896,
  0,
  0,
  0.590461,
  -0.62017,
  0.42122,
  0,
  0.732301,
  -0.915224,
  0,
  -0.999682,
  0,
  -0.613389,
  -0.613389,
  -0.613389,
  -0.613389,
  0,
  0.732301,
  -0.915224,
  0,
  -0.999682,
  0,
  -0.915224,
  0],
 [0,
  -0.815836,
  -0.359449,
  -0.962048,
  0.956868,
  -0.800306,
  0,
  0.478253,
  -0.9

In [72]:
def mean_without_zero(arr):
  arr = [x for x in arr if x != 0]
  return np.mean(arr)


In [73]:
df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))

In [74]:
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,-0.184570,-0.160314,-0.188194,0.8804,0.018444,0.043304,-0.953730,-0.549689,-0.641304
SW,-0.204813,-0.150643,-0.214072,0.9737,0.055146,0.129474,-0.976958,-0.243991,-0.406652
CO,-0.184570,-0.160314,-0.188194,0.9150,0.018444,0.043304,-0.969718,-0.542591,-0.633023
LM,-0.186952,-0.169573,-0.199064,0.9272,0.029504,0.069270,-0.945934,-0.531226,-0.619763
SW-CO,-0.204813,-0.150643,-0.214072,0.9780,0.055146,0.129474,-0.984243,-0.178609,-0.367724
SW-LM,-0.209925,-0.159902,-0.227230,0.9673,0.047613,0.111787,-0.977691,-0.247809,-0.413016
CO-LM,-0.186952,-0.169573,-0.199064,0.9449,0.029504,0.069270,-0.965830,-0.535354,-0.610055
SW-CO-LM,-0.209925,-0.159902,-0.227230,0.9731,0.047613,0.111787,-0.984533,-0.195884,-0.403290


Kinda done with this part. Looks okay for now. Now we got to see if we can get an mp3 version of the song and see if the song sounds sad or happy. 